# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication. **Please note that the "metadata" section must be filled as shown, including the ResourceId that must contain the connection name reporten on line 1**:
```
name: mmai-hub04-fvye-connection-AISearch
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net/
is_shared: true
metadata:
  ApiType: Azure
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-fvye-connection-AISearch
  type: azure_ai_search
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`

# Constants

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
print("Environment variables have been loaded ;-)")

deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME_MINI"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')
version = importlib.metadata.version("azure-ai-projects")
print(f"azure-ai-projects library installed version: {version}")

Project Connection String: <...hub01-grp;mmai-swc-hub01-prj01>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai-swc-hub01-grp',
 'project_name': 'mmai-swc-hub01-prj01'}

# List existing connections and highlight the "AI Search" ones

In [3]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

{
 "name": "mmai-swc-hub01-oais581696736083_aoai",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai-swc-hub01-grp/providers/Microsoft.MachineLearningServices/workspaces/mmai-swc-hub01-prj01/connections/mmai-swc-hub01-oais581696736083_aoai",
 "authentication_type": "ApiKey",
 "connection_type": "ConnectionType.AZURE_OPEN_AI",
 "endpoint_url": "https://mmai-swc-hub01-oais581696736083.openai.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "mmai-swc-hub01-oais581696736083",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai-swc-hub01-grp/providers/Microsoft.MachineLearningServices/workspaces/mmai-swc-hub01-prj01/connections/mmai-swc-hub01-oais581696736083",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_AI_SERVICES",
 "endpoint_url": "https://mmai-swc-hub01-oais581696736083.cognitiveservices.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "mmaihub04groundingwithbingsearch

# Chosen AI Search Connection (just for confirmation)

In [4]:
conn_id

'/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai-swc-hub01-grp/providers/Microsoft.MachineLearningServices/workspaces/mmai-swc-hub01-prj01/connections/mmaiswchub01search01'

# Initialize `AzureAISearchTool`, adding the connection id to it

In [5]:
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

ai_search.definitions: [{'type': 'azure_ai_search'}]
ai_search.resources: {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai-swc-hub01-grp/providers/Microsoft.MachineLearningServices/workspaces/mmai-swc-hub01-prj01/connections/mmaiswchub01search01', 'index_name': 'ms-surface-specs-index', 'query_type': 'simple', 'filter': '', 'top_k': 5}]}}


# Create AI Foundry Agent

In [6]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="aisearch-agent",
    instructions="You are a helpful assistant. Do **NOT** you your internal knowledge, please just search within the given search engine.",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_IYs1yCZQyRPFXgtpfCrpnGoL', 'object': 'assistant', 'created_at': 1745739963, 'name': 'aisearch-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant. Do **NOT** you your internal knowledge, please just search within the given search engine.', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai-swc-hub01-grp/providers/Microsoft.MachineLearningServices/workspaces/mmai-swc-hub01-prj01/connections/mmaiswchub01search01', 'index_name': 'ms-surface-specs-index', 'query_type': 'simple', 'top_k': 5, 'filter': ''}]}}, 'metadata': {}, 'response_format': 'auto'}>

# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="In the documentation I've given you, what kind of keys does the MS Surface Book keyboard include?",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_PydvJMyU9IotOZ4n06HXNJe6', 'object': 'thread', 'created_at': 1745739964, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_ngf6xqKQ8QCSX6QtJA1z8jTT', 'object': 'thread.message', 'created_at': 1745739964, 'assistant_id': None, 'thread_id': 'thread_PydvJMyU9IotOZ4n06HXNJe6', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "In the documentation I've given you, what kind of keys does the MS Surface Book keyboard include?", 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously
Please ensure that the project managed identity has 
- Search Index Data Reader
- Search Service Contributor
<br/>
roles on the Search resource, and the Search resource allows role-based access.

In [8]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_Do9puxajJwK2IHMaJ9vCrQo5', 'object': 'thread.run', 'created_at': 1745739965, 'assistant_id': 'asst_IYs1yCZQyRPFXgtpfCrpnGoL', 'thread_id': 'thread_PydvJMyU9IotOZ4n06HXNJe6', 'status': 'completed', 'started_at': 1745739965, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1745739968, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant. Do **NOT** you your internal knowledge, please just search within the given search engine.', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 2933, 'completion_tokens': 162, 'total_tokens': 3095, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'to

# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: In the documentation I've given you, what kind of keys does the MS Surface Book keyboard include?

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: The Microsoft Surface Book keyboard includes the following types of keys:

1. **Standard Layout Keys**: It features a standard keyboard layout.
2. **Backlit Keys**: The keys are backlit for use in low-light conditions.
3. **Shortcut and Function Keys**: These help in performing common tasks quickly. Keys on the top row also double as function keys, activated by holding the Fn key.
4. **Special Fn Key Combinations**: The Fn key can be used with other keys for specific functions, such as controlling keyboard backlighting, media (play/pause), and adjusting volume (mute, volume up, and volume down)【3:0†source】【3:1†source】.
>>> Annotation in MessageTextContent 1 of message 2: 【3:0†source】

>>> Annotation in MessageTextContent 1 of message 2:

In [10]:
# Get the last message from the sender
last_msg = messages.get_last_message_by_role("assistant")
if last_msg:
    print(last_msg.content[0].text.value)

The Microsoft Surface Book keyboard includes the following types of keys:

1. **Standard Layout Keys**: It features a standard keyboard layout.
2. **Backlit Keys**: The keys are backlit for use in low-light conditions.
3. **Shortcut and Function Keys**: These help in performing common tasks quickly. Keys on the top row also double as function keys, activated by holding the Fn key.
4. **Special Fn Key Combinations**: The Fn key can be used with other keys for specific functions, such as controlling keyboard backlighting, media (play/pause), and adjusting volume (mute, volume up, and volume down)【3:0†source】【3:1†source】.


# Print annotations from the messages

In [11]:
print(f"Number of annotation(s): {len(last_msg.content[0].text.annotations)}")

for annotation in last_msg.content[0].text.annotations:
    print(annotation["text"], annotation["url_citation"]["url"])

Number of annotation(s): 2
【3:0†source】 doc_0
【3:1†source】 doc_1


# Fetch citations from the messages

In [12]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path citations: {len(messages.file_citation_annotations)}\n")

i=0
for file_citation_annotation in messages.file_citation_annotations:
    i += 1
    print(f"{i} - File file_citation_annotation: {file_citation_annotation}")

Nr. of file path citations: 0



# Retrieve and download eventual images

In [13]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path annotations: {len(messages.file_path_annotations)}\n")

i=0
for file_path_annotation in messages.file_path_annotations:
    i += 1
    print(f"{i} - File annotation paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}\\{file_name}")
    image = Image.open(f"{os.getcwd()}\\{file_name}") # Open the image
    image.show() # Display the image

Nr. of file path annotations: 0



# Run Steps

In [14]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_1qfP8kGEwzn1qXii0Hb1Wpaa', 'object': 'thread.run.step', 'created_at': 1745739966, 'run_id': 'run_Do9puxajJwK2IHMaJ9vCrQo5', 'assistant_id': 'asst_IYs1yCZQyRPFXgtpfCrpnGoL', 'thread_id': 'thread_PydvJMyU9IotOZ4n06HXNJe6', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1745739968, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_vOsjEzaGzBx60ii3cEHr4pBM'}}, 'usage': {'prompt_tokens': 2574, 'completion_tokens': 147, 'total_tokens': 2721, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_JHRQYWIiz8FcnkZ4imNO4woM', 'object': 'thread.run.step', 'created_at': 1745739966, 'run_id': 'run_Do9puxajJwK2IHMaJ9vCrQo5', 'assistant_id': 'asst_IYs1yCZQyRPFXgtpfCrpnGoL', 'thread_id': 'thread_PydvJMyU9IotOZ4n06HXNJe6', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at':

# Collect all resources for this project

In [15]:
all_agents = list_all_agents(project_client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(project_client)
print(all_threads["summary"])

all_files = list_all_files(project_client)
print(all_files["summary"])

all_runs = list_all_runs(project_client)
print(all_runs["summary"])

all_runsteps=list_all_runsteps(project_client)
print(all_runsteps["summary"])

all_messages = list_all_messages(project_client)
print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(project_client=project_client)
print(all_vectorstores["summary"])

1 agents in project <mmai-swc-hub01-prj01>
1 threads in project <mmai-swc-hub01-prj01>
0 files in project <mmai-swc-hub01-prj01>
1 runs in 1 threads of project <mmai-swc-hub01-prj01>
2 run steps in 1 pairs of (thread, run) of project <mmai-swc-hub01-prj01>
2 messages in 1 threads of project <mmai-swc-hub01-prj01>
0 vector stores in project <mmai-swc-hub01-prj01>


# Teardown for all resources

In [16]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]["data"]:
    i += 1
    project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(project_client=project_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [17]:
# delete all files

i=0
for file in all_files['content']['data']:
    i += 1
    project_client.agents.delete_file(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [18]:
# delete all threads

i=0
for thread in all_threads["content"]["data"]:
    i += 1
    project_client.agents.delete_thread(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_PydvJMyU9IotOZ4n06HXNJe6> has been deleted
Threads deleted: 1



In [19]:
# delete all agents

i=0
for agent in all_agents["content"]["data"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(project_client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_IYs1yCZQyRPFXgtpfCrpnGoL> has been deleted
Agents deleted: 1



# HIC SUNT LEONES